In [1]:
import pandas as pd
import re
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.preprocessing import StandardScaler, LabelEncoder, MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_percentage_error
import wandb
import torch
import torch.nn as nn
import torch.optim as optim
import tensorflow as tf
from tensorflow.keras.losses import MeanSquaredLogarithmicError
from tensorflow.keras.optimizers import Adam


In [2]:
data_path = "SoftwareEngineer_cleaned.csv"
df = pd.read_csv(data_path)
print(df.shape)
df.head()


(34012, 12)


,title,level,company,yearsofexperience,totalyearlycompensation,gender,Doctorate_Degree,Masters_Degree,Bachelors_Degree,tag,state,city
0,Software Engineer,SE 2,eBay,5.0,100000,NaN,0,0,0,NaN,CA,San Francisco
1,Software Engineer,60,Microsoft,5.0,157000,NaN,0,0,0,NaN,CA,Mountain View
2,Software Engineer,63,Microsoft,8.5,208000,NaN,0,0,0,NaN,WA,Seattle
3,Software Engineer,62,Microsoft,4.0,156000,NaN,0,0,0,NaN,WA,Seattle
4,Software Engineer,59,Microsoft,3.0,120000,NaN,0,0,0,NaN,WA,Redmond


In [3]:
headers1 = ["company", "yearsofexperience", "Doctorate_Degree", "Masters_Degree", "Bachelors_Degree", "state", "city", "tag", "gender", 'level', "totalyearlycompensation"]
cat = ["company", "state", "city", "tag", "gender", 'level']
df_clean = df[headers1]
df_clean

,company,yearsofexperience,Doctorate_Degree,Masters_Degree,Bachelors_Degree,state,city,tag,gender,level,totalyearlycompensation
0,eBay,5.0,0,0,0,CA,San Francisco,NaN,NaN,SE 2,100000
1,Microsoft,5.0,0,0,0,CA,Mountain View,NaN,NaN,60,157000
2,Microsoft,8.5,0,0,0,WA,Seattle,NaN,NaN,63,208000
3,Microsoft,4.0,0,0,0,WA,Seattle,NaN,NaN,62,156000
4,Microsoft,3.0,0,0,0,WA,Redmond,NaN,NaN,59,120000
...,...,...,...,...,...,...,...,...,...,...,...
34007,Google,10.0,0,0,0,WA,Seattle,Distributed Systems (Back-End),NaN,T4,327000
34008,Microsoft,2.0,0,0,0,WA,Redmond,Full Stack,NaN,62,237000
34009,MSFT,14.0,0,0,0,WA,Seattle,Full Stack,NaN,63,220000
34010,Salesforce,8.0,0,0,0,CA,San Francisco,iOS,NaN,Lead MTS,280000


In [4]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(df_clean[cat]), index=df_clean.index)

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names_out(cat)
encode_df.head()

C:\Users\pedia\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:808: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


,company_10x Genomics,company_23andMe,company_2U,company_3M,company_3m,company_7-Eleven,company_7-eleven,company_8x8,company_ABB,company_ADP,...,level_unknown,level_vice president,level_x2,level_x4,level_x5,level_x6,level_x7,level_z4,level_||,level_nan
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
df_clean = df_clean.merge(encode_df,left_index=True, right_index=True)
df_clean = df_clean.drop(cat,1)
df_clean# Merge one-hot encoded features and drop the originals
df_clean

C:\Users\pedia\AppData\Local\Temp\ipykernel_27352\1232695819.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_clean = df_clean.drop(cat,1)


,yearsofexperience,Doctorate_Degree,Masters_Degree,Bachelors_Degree,totalyearlycompensation,company_10x Genomics,company_23andMe,company_2U,company_3M,company_3m,...,level_unknown,level_vice president,level_x2,level_x4,level_x5,level_x6,level_x7,level_z4,level_||,level_nan
0,5.0,0,0,0,100000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,5.0,0,0,0,157000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,8.5,0,0,0,208000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4.0,0,0,0,156000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,3.0,0,0,0,120000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34007,10.0,0,0,0,327000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
34008,2.0,0,0,0,237000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
34009,14.0,0,0,0,220000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
34010,8.0,0,0,0,280000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
# Split our preprocessed data into our features and target arrays
y = df_clean.totalyearlycompensation
X = df_clean.drop(columns=["totalyearlycompensation"])

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78, train_size=0.8)

In [7]:
X_train_scaled1 = MinMaxScaler().fit_transform(X_train)
X_test_scaled1 = MinMaxScaler().fit_transform(X_test)
X_train_scaled1

array([[0.51724138, 0.        , 1.        , ..., 0.        , 0.        ,
        0.        ],
       [0.31034483, 0.        , 1.        , ..., 0.        , 0.        ,
        0.        ],
       [0.20689655, 0.        , 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.34482759, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.13793103, 0.        , 1.        , ..., 0.        , 0.        ,
        0.        ],
       [0.27586207, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [8]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [9]:
y_train

11464    420000
27756    236000
11455    185000
18103    250000
16324    250000
          ...  
20558    238000
18547    165000
3476     450000
22633    174000
16854    140000
Name: totalyearlycompensation, Length: 27209, dtype: int64

In [10]:
len(X_train_scaled1[0])

4215

In [11]:
#### Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 4500
hidden_nodes_layer2 = 4500
hidden_nodes_layer3 = 1000
hidden_nodes_layer4 = 300

nn4 = tf.keras.models.Sequential()

# First hidden layer
nn4.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn4.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn4.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# 4th hidden layer
nn4.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="relu"))

# Output layer
nn4.add(tf.keras.layers.Dense(units=1, activation="linear"))

# Check the structure of the model
nn4.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 4500)              18972000  
                                                                 
 dense_1 (Dense)             (None, 4500)              20254500  
                                                                 
 dense_2 (Dense)             (None, 1000)              4501000   
                                                                 
 dense_3 (Dense)             (None, 300)               300300    
                                                                 
 dense_4 (Dense)             (None, 1)                 301       
                                                                 
Total params: 44,028,101
Trainable params: 44,028,101
Non-trainable params: 0
_________________________________________________________________


In [12]:
# Compile the model
msle = MeanSquaredLogarithmicError()
mape = mean_absolute_percentage_error

nn4.compile(loss='mse', optimizer=Adam(learning_rate=0.0001), metrics=[tf.keras.metrics.mean_absolute_percentage_error])

In [13]:
fit_model = nn4.fit(X_train_scaled,y_train,epochs=20,batch_size=100,validation_split=0.2)

C:\Users\pedia\anaconda3\lib\site-packages\keras\engine\data_adapter.py:1696: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  return t[start:end]


Epoch 1/20
218/218 [==============================] - 102s 462ms/step - loss: 29352458240.0000 - mean_absolute_percentage_error: 55.3552 - val_loss: 5943578624.0000 - val_mean_absolute_percentage_error: 23.6659
Epoch 2/20
218/218 [==============================] - 101s 465ms/step - loss: 5574050816.0000 - mean_absolute_percentage_error: 22.1270 - val_loss: 4455252480.0000 - val_mean_absolute_percentage_error: 18.5503
Epoch 3/20
218/218 [==============================] - 97s 446ms/step - loss: 3947942400.0000 - mean_absolute_percentage_error: 18.0290 - val_loss: 3777341184.0000 - val_mean_absolute_percentage_error: 18.2890
Epoch 4/20
218/218 [==============================] - 97s 445ms/step - loss: 3226775552.0000 - mean_absolute_percentage_error: 16.1228 - val_loss: 3502481152.0000 - val_mean_absolute_percentage_error: 17.5593
Epoch 5/20
218/218 [==============================] - 97s 445ms/step - loss: 2814432768.0000 - mean_absolute_percentage_error: 14.9364 - val_loss: 3484937216.000

KeyboardInterrupt: 

In [ ]:
# Generate predictions for the test data
y_pred = nn4.predict(X_test_scaled)

# Compare the predictions to the actual values
predictions_df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred.flatten()})

In [ ]:
predictions_df

In [ ]:
r2_score = r2_score(y_test, y_pred)
print('R2 score:', r2_score)

## Using MSE as loss function
## Finally i achieved R2 Score 73% and MAPE 13% with Z-normalize
## Finally i achieved R2 Score 76% and MAPE 10.8% with MIN-MAX
## Using MAE as loss function
## Finally i achieved R2 Score 76.8% and MAPE 10.4% with MIN-MAX
## Using MAPE as loss function
## Finally i achieved R2 Score 76.9% and MAPE 10.1% with MIN-MAX


In [ ]:
import matplotlib.pyplot as plt

plt.scatter(y_test, y_pred)

# Add a line of best fit
plt.plot(y_test, y_test, color = 'red')

# Add axis labels
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')

# Show the plot
plt.show()

In [ ]:
nn4.save('Saved_Models/NeuralNetwork/NNRPoisson2.h5')

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

model = nn4
k = 5
kf = KFold(n_splits=k, shuffle=True, random_state=42)
# Iterate over the folds
for train_index, test_index in kf.split(X):
    # Split the data into training and testing sets
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    # Train the model on the training data
    model.fit(X_train, y_train)
    
    # Evaluate the model on the testing data
    y_pred = model.predict(X_test)
    score = mean_absolute_percentage_error(y_test, y_pred)
    scores.append(score)

# Compute the mean score and standard deviation
mean_score = np.mean(scores)
std_score = np.std(scores)